**Cargar el dataset y mostrar el resumen de valores faltantes:**

In [ ]:
import pandas as pd
import numpy as np

# Cargar el dataset completo
df = pd.read_csv('houses_medellin.csv')

# Mostrar un resumen de los valores faltantes por columna
print("Resumen de valores faltantes por columna:")
print(df.isnull().sum())


**Visualizar el porcentaje de valores faltantes:**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calcular el porcentaje de valores faltantes y visualizarlo
plt.figure(figsize=(10, 5))
(df.isnull().mean() * 100).sort_values(ascending=False).plot.bar(color='skyblue')
plt.ylabel("Porcentaje de valores faltantes (%)")
plt.title("Valores faltantes por columna")
plt.show()


**Preprocesamiento: imputación y vectorización para TODAS las columnas:**

In [ ]:
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Identificar columnas numéricas y categóricas en todo el DataFrame
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

print("Columnas numéricas:", num_cols)
print("Columnas categóricas:", cat_cols)

# Pipeline para columnas numéricas: imputar valores faltantes con KNNImputer
num_pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5))
])

# Pipeline para columnas categóricas: imputar con 'most_frequent' y codificar con OneHotEncoder
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combinar ambos pipelines en un ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

# Aplicar la transformación al DataFrame completo
X_processed = preprocessor.fit_transform(df)


**Reconstruir el DataFrame final con nuevos nombres de columnas:**

In [ ]:
# Generar nuevos nombres de columnas:
# - Las columnas numéricas mantienen sus nombres originales.
new_num_cols = num_cols

# - Para las columnas categóricas, extraer los nombres generados por OneHotEncoder
ohe = preprocessor.named_transformers_['cat'].named_steps['encoder']
new_cat_cols = [f"{col}_{cat}" for col, cats in zip(cat_cols, ohe.categories_) for cat in cats]

# Combinar ambos conjuntos de nombres
new_columns = new_num_cols + new_cat_cols

# Crear el DataFrame final con los datos procesados
df_processed_final = pd.DataFrame(X_processed, columns=new_columns)


**Mostrar los resultados finales:**

In [ ]:
# Mostrar la forma y una vista previa del DataFrame procesado
print("Forma del DataFrame procesado:", df_processed_final.shape)
print("Vista previa del DataFrame procesado:")
print(df_processed_final.head())